In [171]:
import json
from base64 import b64decode
import f90nml
from collections import OrderedDict
import os

In [172]:
with open('database/espresso/7.2/database.json') as f:
    database = json.load(f)

In [173]:
def nl_print(string):
    if string is True:
        return '.true.'
    elif string is False:
        return '.false.'
    elif isinstance(string, str):
        return f"'{string}'"
    else:
        return string

base_link = "file://" + os.path.join(os.getcwd(), 'database/espresso/7.2/INPUT_PW.html')
input_file = f90nml.read('pw.in')
with_comments = OrderedDict()
html_string = "<div><pre>\n"
preview_string = ""
js_string = """// JavaScript code to show and hide the preview windows
document.addEventListener("DOMContentLoaded", function() {\n\n"""
for nl in input_file.keys():
    html_string += f"&{nl}\n"
    namelist = input_file[nl]
    new_namelist = OrderedDict()
    for tag, val in namelist.items():
        idm = database[nl][tag]['idm']
        html = database[nl][tag]['html']
        html = b64decode(html.encode('utf-8')).decode('utf-8')
        link = base_link+"#"+idm
        tag_html = f'<a href="{link}" id="{tag}">{tag}</a>'

        if database[nl][tag]['options']:
            options = database[nl][tag]['options']
            comment = options.get(str(val), None)
            if comment is None:
                comment = "Unknown Value"
        elif database[nl][tag]['info']:
            comment = database[nl][tag]['info']
        else:
            comment = ''
        comment = ' ! ' + comment if comment else ''
        comment = comment.split('.')[0]
        comment = comment.split('-')[0]
        comment = comment.split('(')[0]
        comment = comment.split('see')[0]
        comment = comment.split(':')[0]

        if isinstance(val, list):
            for i, v in enumerate(val):
                html_string += f"  {tag_html}({i+1}) = {nl_print(v)}{comment}\n"
        else:
            html_string += f"  {tag_html} = {nl_print(val)}{comment}\n"
        js_string += f'var link_{tag} = document.getElementById("{tag}");\n'
        js_string += f'var preview_{tag} = document.getElementById("preview_{tag}");\n'
        js_string += f'link_{tag}.addEventListener("mouseover", function() {{ preview_{tag}.style.display = "block"; }});\n'
        js_string += f'link_{tag}.addEventListener("mouseout", function() {{ preview_{tag}.style.display = "none"; }});\n\n'

        preview_link = "file:///" + os.path.join(os.getcwd(), f'work/webpages/{tag}.html')
        preview_string += f'<div class="preview" id="preview_{tag}">\n'
        preview_string += f'<object data="{preview_link}" width="100%" height="100%" type="text/html">'
        preview_string += ' </object>\n</div>\n\n'

    html_string += "/\n"
html_string += "</pre></div>\n" + preview_string
js_string += "});"

In [174]:
from lxml.html import fragment_fromstring, fromstring
from lxml.etree import tostring

webpage_template = """
<!DOCTYPE html>
<html>
<head>
  <style>
    /* Style for the preview window */
    .preview {
      position: absolute;
      display: none;
      width: 400px;
      height: 300px;
      border: 1px solid #ccc;
      background-color: white;
      padding: 10px;
    }
  </style>
</head>
<body>
</body>
</html>
"""
webpage = fromstring(webpage_template)
body = webpage.xpath("//body")[0]
html = fromstring(html_string)
body.insert(0, html)

head = webpage.xpath("//head")[0]
js = fromstring(f'<script type="text/javascript">\n{js_string}\n</script>')
head.insert(0, js)

with open('webpage.html', 'w') as f:
    f.write(tostring(webpage, pretty_print=True).decode('utf-8'))


In [175]:
print(preview_string)

<div class="preview" id="preview_calculation">
<object data="file:////Users/ashour/code/dft-tutor/work/webpages/calculation.html" width="100%" height="100%" type="text/html"> </object>
</div>

<div class="preview" id="preview_etot_conv_thr">
<object data="file:////Users/ashour/code/dft-tutor/work/webpages/etot_conv_thr.html" width="100%" height="100%" type="text/html"> </object>
</div>

<div class="preview" id="preview_forc_conv_thr">
<object data="file:////Users/ashour/code/dft-tutor/work/webpages/forc_conv_thr.html" width="100%" height="100%" type="text/html"> </object>
</div>

<div class="preview" id="preview_outdir">
<object data="file:////Users/ashour/code/dft-tutor/work/webpages/outdir.html" width="100%" height="100%" type="text/html"> </object>
</div>

<div class="preview" id="preview_prefix">
<object data="file:////Users/ashour/code/dft-tutor/work/webpages/prefix.html" width="100%" height="100%" type="text/html"> </object>
</div>

<div class="preview" id="preview_pseudo_dir">
<